# Contract Value

In [18]:
# basic libaries 
import pandas as pd
import numpy as np
import datetime

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Just personal preference of colours 
Kookaburra = "#003C55"
Possum = "#73234B"
DarkBlue = "061E3E"
Blue = "002960"
Red = "CD202C"
Turquoise = "006983"
Cyan = "0093D3"

# Million
Million = 1000000

In [19]:
def count(lst, target):
    
    """
    This function counts the number of target objects in the given list.
    """
    
    num_target = 0 
    
    for i in lst:
        if i == target:
            num_target += 1
            
    return num_target 

def convert_date(Date):
    
    """
    The function converts the input date to its datetime format.
    The input must be string written as "YYYY/MM/DD"
    """
    
    Y = int(Date.split("/")[0])
    M = int(Date.split("/")[1])
    D = int(Date.split("/")[2])
    
    return datetime.date(Y, M, D)


def delta_day(Date1, Date2):
    
    """This function returns the difference of days between the two input dates
    The input must be string written as "YYYY/MM/DD"""
    
    delta = convert_date(Date1) - convert_date(Date2)
    
    return delta.days 
    

In [104]:
def Pricing_Model(In_dates, With_dates, Prices, Rate, Max_vol, Storage):
    
    """
    
    Parameters: 
        In_dates: a list of multiple injection dates
        With_dates: a list of multiple withdrawl dates
        Prices: a dictionary that contains historic natural gas prices as $X per ton. 
            It should follow the format: {<YYYY/MM/DD>: <price>,
                                        {<YYYY/MM/DD>: <price>}
            We neeed to ensure that each of In_dates and With_dates has a corresponding price in the dictionary, 
            or it will return an error. 

        Rate: a constant injection or withdrawl rate, written as $X per million ton
        Max_vol: maximum volumns to be stored, X million tons
        Storage: storate cost per million tons per month
        
    """
    
    # Initiate the costs
    Current_storage = 0
    val_contract = 0
    Storage_cost = 0
    
    # Create a list of dates that contain all injection and withdrawal dates without repetition
    combined_dates = set(In_dates + With_dates)
    combined_dates = sorted(list(set(In_dates + With_dates)))
    
    # Calculate the transaction cost
    Transaction_cost = Rate * (len(In_dates) + len(With_dates))
    
    #print("combined dates = ", combined_dates)
    
    # Iterate through the whole list of dates to whether it has injection or withdrawl activities 
    for i in range(len(combined_dates)):
        
        date = combined_dates[i] # Current date
    
        # If the current date is part of the injection dates, we buy the assets
        if date in In_dates:
            val_contract -= count(In_dates, date)* Prices[date] * Million
            # We count how many injections happen at the day
        
        # If the current date is part of the withdraw dates, we sell the assets
        elif date in With_dates:
            val_contract += count(With_dates, date)* Prices[date] * Million
            # We count how many withdrawls happen at the day
        
        # Now we can have the current storage amount 
        Current_storage += count(In_dates, date) - count(With_dates, date)
        
        # If the contract storage at any point reaches exceeds its maximum volume,\
        # we need to message the client to consider another transaction
        
        if Current_storage > Max_vol:
            print("This contract cannot happen as the storage exceeds its maximum volume, please consider another transaction")
            return 0 
        
        # Only when we are not in the last date should we consider next activity to get our storage cost
        if i < len(combined_dates) - 1:
            next_date = combined_dates[i + 1]
            
            Storage_cost += Current_storage * Storage * delta_day(next_date, date) / 365
            
    val_contract -= (Storage_cost + Transaction_cost)
    
    return val_contract
    


### Testing Part 

In [101]:
In_dates = ["2023/02/03", "2023/05/06", "2023/05/06", "2023/02/03", "2023/02/05"]
With_dates = ["2023/04/03", "2023/12/06", "2023/05/07"]
Prices = {"2023/02/03":10.2,
          "2023/02/05":11.5,
         "2023/05/06": 11.2,
         "2023/04/03": 13.7,
         "2023/05/07": 11.6,
         "2023/12/06":7.8,
         "2023/04/08": 7.5}
Rate = 10000
Max_vol = 2
Storage = 100000

In [105]:
Pricing_Model(In_dates, With_dates, Prices, Rate, Max_vol, Storage)

This contract cannot happen as the storage exceeds its maximum volume, please consider another transaction


0